<a href="https://colab.research.google.com/github/exsantucho/AnalisisYCuracion/blob/master/Entregable_Parte_1_V00_E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exploración y Curación de Datos**

*Edición 2021*

----

# Trabajo práctico entregable - parte 1

En esta notebook, vamos a cargar el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

In [142]:
import matplotlib.pyplot as plt
import numpy
import pandas

import seaborn
seaborn.set_context('talk')

In [143]:
import plotly
plotly.__version__
# Make sure it's 4.14.3

'4.14.3'

In [144]:
# To update plotly, uncomment and run the following line:
# !pip install plotly --upgrade

In [145]:
melb_df = pandas.read_csv('https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')

mebl_df_respaldo = melb_df
melb_df[:10]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0
5,Abbotsford,129 Charles St,2,h,941000.0,S,Jellis,7/05/2016,2.5,3067.0,2.0,1.0,0.0,181.0,NaN,NaN,Yarra,-37.8041,144.9953,Northern Metropolitan,4019.0
6,Abbotsford,124 Yarra St,3,h,1876000.0,S,Nelson,7/05/2016,2.5,3067.0,4.0,2.0,0.0,245.0,210.0,1910.0,Yarra,-37.8024,144.9993,Northern Metropolitan,4019.0
7,Abbotsford,98 Charles St,2,h,1636000.0,S,Nelson,8/10/2016,2.5,3067.0,2.0,1.0,2.0,256.0,107.0,1890.0,Yarra,-37.8060,144.9954,Northern Metropolitan,4019.0
8,Abbotsford,6/241 Nicholson St,1,u,300000.0,S,Biggin,8/10/2016,2.5,3067.0,1.0,1.0,1.0,0.0,NaN,NaN,Yarra,-37.8008,144.9973,Northern Metropolitan,4019.0
9,Abbotsford,10 Valiant St,2,h,1097000.0,S,Biggin,8/10/2016,2.5,3067.0,3.0,1.0,2.0,220.0,75.0,1900.0,Yarra,-37.8010,144.9989,Northern Metropolitan,4019.0


## Ejercicio 1: 

1. Eliminar los valores extremos que no sean relevantes para la predicción de valores de las propiedades.

1. Investigar la distribución de las variables del conjunto de datos y seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad.
  1. Justificar cada columna no seleccionada.
  2. Para las variables categóricas seleccionadas, agrupe o combine las categorías poco frecuentes para asegurar que todos los grupos tengan un número mínimo de registros.

2. Agregar información adicional respectiva al entorno de una propiedad a partir del [conjunto de datos de AirBnB](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) utilizado en el práctico. 
  1. Seleccionar qué variables agregar y qué combinaciones aplicar a cada una. Por ejemplo, pueden utilizar solo la columna `price`, o aplicar múltiples transformaciones como la mediana o el mínimo.
  1. Utilizar la variable zipcode para unir los conjuntos de datos. Sólo incluir los zipcodes que tengan una cantidad mínima de registros (a elección) como para que la información agregada sea relevante.
  2. Investigar al menos otras 2 variables que puedan servir para combinar los datos, y justificar si serían adecuadas o no. Pueden asumir que cuentan con la ayuda de anotadores expertos para encontrar equivalencias entre barrios o direcciones, o que cuentan con algoritmos para encontrar las n ubicaciones más cercanas a una propiedad a partir de sus coordenadas geográficas. **NO** es necesario que realicen la implementación.

Pueden leer otras columnas del conjunto de AirBnB además de las que están en `interesting_cols`, si les parecen relevantes.


###Ejercicio 1.1###

In [146]:
print('Cantidad de registros: ', len(melb_df))
# Descripción del df
melb_df.describe().loc[['count','mean','min','max']].round(2)

Cantidad de registros:  13580


,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.00,13580.00,13580.00,13580.0,13580.00,13580.00,13518.00,13580.00,7130.00,8205.00,13580.00,13580.00,13580.00
mean,2.94,1075684.08,10.14,3105.3,2.91,1.53,1.61,558.42,151.97,1964.68,-37.81,145.00,7454.42
min,1.00,85000.00,0.00,3000.0,0.00,0.00,0.00,0.00,0.00,1196.00,-38.18,144.43,249.00
max,10.00,9000000.00,48.10,3977.0,20.00,8.00,10.00,433014.00,44515.00,2018.00,-37.41,145.53,21650.00


In [147]:
# Conteo de valores por columnas
melb_df['Rooms'].value_counts()
melb_df['Bedroom2'].value_counts()
melb_df['Bathroom'].value_counts()
melb_df['Car'].value_counts()
len(melb_df[melb_df['BuildingArea']>0])

7113

In [148]:
# Criterios para depuración del df. Se mantienen los registros que cumplan con:
# 1) Cantidad razonable (máxima y mínima) de ambientes: Rooms, Bedroom2, Bathroom, Car.
# 2) Cantidad de habitaciones (Rooms) mayor o igual a cantidad de dormitorios (Bedroom2).
# 3) Area construida (BuildingArea) distinta de cero (incluirá valores mayores a cero y NaN. Luego se verá cómo imputar valores a NaN).
# 4) Año de construcción (YearBuilt) mayor a valor fijado.

# 1)
lim_rooms = 5
lim_bedrooms2 = 5
lim_bathroom = 4
lim_car = 4

melb_df = melb_df[(melb_df['Rooms']>0) & (melb_df['Rooms']<=lim_rooms)]
melb_df = melb_df[(melb_df['Bedroom2']>0) & (melb_df['Bedroom2']<=lim_bedrooms2)]
melb_df = melb_df[(melb_df['Bathroom']>0) & (melb_df['Bathroom']<=lim_bathroom)]
melb_df = melb_df[melb_df['Car']<=lim_car]
melb_df.describe().loc[['count','mean','min','max']].round(2)

# 2)
melb_df = melb_df[melb_df['Rooms'] >= melb_df['Bedroom2']]
melb_df.describe().loc[['count','mean','min','max']].round(2)

# 3)
melb_df = melb_df[melb_df['BuildingArea']!=0]
melb_df.describe().loc[['count','mean','min','max']].round(2)

# 4)
lim_year_built = 1800
melb_df = melb_df[melb_df['YearBuilt']>=lim_year_built]
melb_df.describe().loc[['count','mean','min','max']].round(2)

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,7888.00,7888.00,7888.00,7888.0,7888.00,7888.00,7888.00,7888.00,6590.00,7888.00,7888.00,7888.00,7888.00
mean,2.91,1048409.38,10.01,3105.2,2.87,1.55,1.54,488.34,141.06,1965.04,-37.81,144.99,7481.82
min,1.00,85000.00,0.00,3000.0,1.00,1.00,0.00,0.00,1.00,1830.00,-38.16,144.54,389.00
max,5.00,9000000.00,47.40,3977.0,5.00,4.00,4.00,37000.00,3112.00,2017.00,-37.41,145.53,21650.00


###Ejercicio 1.2###

In [149]:
print(melb_df.columns)
interesting_melb_cols = ['Rooms', 'Type', 'Price', 'Distance',
                    'Bedroom2', 'Bathroom', 'Car', 'Landsize',
                    'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude']

# Justificación de columnas eliminadas:
# Suburb: Dato considerado indirectamente con 'Distance' y 'Lattitude', 'Longtitude'
# Address: Idem Suburb
# Method: No interesa la condición de venta previa y/o tenencias
# SellerG: No importa la agencia de venta
# Date: La fecha de venta no interesa, sino que se resume con año de construcción
# Postcode: Idem Suburb
# CouncilArea: Idem Suburb
# Regionname: Idem Suburb
# Propertycount: Idem Suburb

melb_df = melb_df.loc[:, interesting_melb_cols]
melb_df.head(3)

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')


,Rooms,Type,Price,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude
1,2,h,1035000.0,2.5,2.0,1.0,0.0,156.0,79.0,1900.0,-37.8079,144.9934
2,3,h,1465000.0,2.5,3.0,2.0,0.0,134.0,150.0,1900.0,-37.8093,144.9944
4,4,h,1600000.0,2.5,3.0,1.0,2.0,120.0,142.0,2014.0,-37.8072,144.9941


### Ejercicio 1.3 ###

In [151]:
# data source:
# https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv

# airbnb_df = pandas.read_csv('https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv')
# print(airbnb_df.columns)

interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
  'property_type', 'bathrooms', 'bedrooms']

airbnb_df = pandas.read_csv('https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv',
            usecols=interesting_cols)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [152]:
airbnb_df[:3]

,description,neighborhood_overview,street,neighborhood,city,suburb,state,zipcode,latitude,longitude,property_type,bathrooms,bedrooms,price,weekly_price,monthly_price
0,"House: Clean, New, Modern, Quite, Safe. 10Km f...",Very safe! Family oriented. Older age group.,"Bulleen, VIC, Australia",Balwyn North,Manningham,Bulleen,VIC,3105,-37.772684,145.092133,House,1.0,1.0,60,NaN,NaN
1,A large air conditioned room with queen spring...,This hip area is a crossroads between two grea...,"Brunswick East, VIC, Australia",Brunswick,Moreland,Brunswick East,VIC,3057,-37.766505,144.980736,Apartment,1.0,1.0,35,200.0,803.0
2,RIGHT IN THE HEART OF ST KILDA! It doesn't get...,A stay at our apartment means you can enjoy so...,"St Kilda, VIC, Australia",St Kilda,Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,Apartment,1.0,1.0,159,1253.0,4452.0


## Ejercicio 2: Imputación

1. Imputar los valores faltantes de la columna CouncilArea en base a la información presente en Suburb. (Si en el punto anterior no seleccionó ninguna de estas variables, agréguelas ahora el conjunto de datos).

2. Imputar los valores faltantes de la(s) columna(s) que se agregaron a partir del conjunto de datos de AirBnB.



## Ejercicio 3

Crear y guardar un nuevo conjunto de datos con todas las transformaciones realizadas anteriormente.